In [1]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Default GPU Device: /device:GPU:0


In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14669225555728055156
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3194762035
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9964983267587210459
physical_device_desc: "device: 0, name: GeForce GTX 960M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [5]:
with tf.device('/device:GPU:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    r = sess.run(c)
    print(r)
    import numpy as np
    assert np.all(r == np.array([[22., 28.], [49., 64.]]))

[[22. 28.]
 [49. 64.]]


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

Using TensorFlow backend.


In [3]:
classifier = Sequential()

W0709 02:51:19.843217 14228 deprecation_wrapper.py:119] From C:\Users\ASUS\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [4]:
classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

W0709 02:51:20.076921 14228 deprecation_wrapper.py:119] From C:\Users\ASUS\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0709 02:51:20.079913 14228 deprecation_wrapper.py:119] From C:\Users\ASUS\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [5]:
classifier.add(MaxPooling2D(pool_size = (2, 2)))

W0709 02:51:20.271447 14228 deprecation_wrapper.py:119] From C:\Users\ASUS\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [6]:
classifier.add(Convolution2D(32, (3, 3), activation = 'relu'))

In [7]:
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [8]:
classifier.add(Flatten())

In [9]:
classifier.add(Dense(activation="relu", units=128))

In [10]:
classifier.add(Dense(activation="sigmoid", units=1))

In [11]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

W0709 02:51:22.300076 14228 deprecation_wrapper.py:119] From C:\Users\ASUS\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0709 02:51:22.326970 14228 deprecation_wrapper.py:119] From C:\Users\ASUS\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0709 02:51:22.332960 14228 deprecation.py:323] From C:\Users\ASUS\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [13]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


In [15]:
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [16]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [17]:
classifier.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 25,
                         validation_data = test_set,
                         nb_val_samples = 2000)

C:\Users\ASUS\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
C:\Users\ASUS\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=250, epochs=25, validation_steps=2000)`
  """
W0709 02:51:29.094136 14228 deprecation_wrapper.py:119] From C:\Users\ASUS\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is depre

Epoch 1/25
250/250 [==============================] - 95s 380ms/step - loss: 0.6610 - acc: 0.5896 - val_loss: 0.6088 - val_acc: 0.6705
Epoch 2/25
250/250 [==============================] - 87s 347ms/step - loss: 0.6024 - acc: 0.6689 - val_loss: 0.6418 - val_acc: 0.6480
Epoch 3/25
250/250 [==============================] - 86s 345ms/step - loss: 0.5663 - acc: 0.7084 - val_loss: 0.5442 - val_acc: 0.7231
Epoch 4/25
250/250 [==============================] - 84s 338ms/step - loss: 0.5268 - acc: 0.7319 - val_loss: 0.5150 - val_acc: 0.7471
Epoch 5/25
250/250 [==============================] - 87s 347ms/step - loss: 0.5084 - acc: 0.7480 - val_loss: 0.5004 - val_acc: 0.7606
Epoch 6/25
250/250 [==============================] - 96s 383ms/step - loss: 0.4787 - acc: 0.7675 - val_loss: 0.5189 - val_acc: 0.7492
Epoch 7/25
250/250 [==============================] - 104s 416ms/step - loss: 0.4704 - acc: 0.7724 - val_loss: 0.4847 - val_acc: 0.7764
Epoch 8/25
250/250 [==============================] - 

In [18]:
from keras.preprocessing import image

In [26]:
inp=input('image dir : ')
test_image = image.load_img('dataset/inputs/'+inp,target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
pred = classifier.predict(test_image)
print(training_set.class_indices)

image dir : 1.jpg
{'cats': 0, 'dogs': 1}


In [27]:
if pred[0][0] == 1:
    print('Dog')
else:
    print('Cat')

Cat
